<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Importar os módulos

In [1]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import Dataset_transf as dprep
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

# Obter os dataframes dos ficheiros

In [2]:
failures_df, signals_df, logs_df = dprep.get_data()

In [3]:
df_list = [failures_df, signals_df, logs_df]

# Criação de datasets de components

In [4]:
df_generator, df_gen_bear, df_transformer, df_hydraulic, df_gearbox = dprep.component_df_creation(signals_df)

In [5]:
comp_df_list = [df_generator, df_hydraulic, df_gen_bear, df_transformer,  df_gearbox]

In [6]:
for i in comp_df_list:
    print(i.shape)

(434145, 45)
(434145, 31)
(434145, 45)
(434145, 36)
(434145, 33)


In [7]:
component_list = failures_df.Component.unique()
component_list

array(['GENERATOR', 'HYDRAULIC_GROUP', 'GENERATOR_BEARING', 'TRANSFORMER',
       'GEARBOX'], dtype=object)

# Merge dos DF com o DF de failures

In [8]:
for i in range(len(component_list)):
    comp_df_list[i] = dprep.sig_fail_merge_dfs(sig_df=comp_df_list[i],
                                               fail_df=failures_df,
                                               component=component_list[i])

In [9]:
for i in comp_df_list:
    print(i.shape)

(434147, 46)
(434145, 32)
(434145, 46)
(434145, 37)
(434145, 34)


# Fillna by turbine

In [10]:
turbine_list = signals_df.Turbine_ID.unique()
turbine_list

array(['T11', 'T06', 'T01', 'T09', 'T07'], dtype=object)

In [11]:
for i in range(len(component_list)):
    comp_df_list[i] = dprep.fill_na_by_turbine(comp_df_list[i], turbine_list)

/Users/miguelamaro/code/miguelamaro4/Final_Project/wmillfailprev/notebooks/Dataset_transf.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  index['date'] = index['Timestamp']
/Users/miguelamaro/code/miguelamaro4/Final_Project/wmillfailprev/notebooks/Dataset_transf.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  index['date'] = index['Timestamp']
/Users/miguelamaro/code/miguelamaro4/Final_Project/wmillfailprev/notebooks/Dataset_transf.py:162: SettingWithCopyWarning: 
A value is trying to be set on 

/Users/miguelamaro/code/miguelamaro4/Final_Project/wmillfailprev/notebooks/Dataset_transf.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  index['date'] = index['Timestamp']
/Users/miguelamaro/code/miguelamaro4/Final_Project/wmillfailprev/notebooks/Dataset_transf.py:162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged['date'] = df_merged['Timestamp']
/Users/miguelamaro/code/miguelamaro4/Final_Project/wmillfailprev/notebooks/Dataset_transf.py:163: SettingWithCopyWarning: 
A value is trying to be

In [12]:
for i in comp_df_list:
    print(i.shape)

(434147, 47)
(434145, 33)
(434145, 47)
(434145, 38)
(434145, 35)


# Criação de Variável alvo

In [13]:
for i in range(len(comp_df_list)):
    print(comp_df_list[i].isna().sum().value_counts())

0    47
dtype: int64
0    33
dtype: int64
0    47
dtype: int64
0    38
dtype: int64
0    35
dtype: int64


In [14]:
days_list = [60, 45, 40, 30, 10, 5]

In [15]:
for day in days_list:
    for i in range(len(comp_df_list)):
        comp_df_list[i] = dprep.aplic_var_target(comp_df_list[i], day)

In [16]:
for i in comp_df_list:
    print(i.shape)

(434147, 53)
(434145, 39)
(434145, 53)
(434145, 44)
(434145, 41)


# Feature selection

In [17]:
for i in comp_df_list:
    print(i.keys())

Index(['Timestamp', 'Turbine_ID', 'Gen_RPM_Max', 'Gen_RPM_Min', 'Gen_RPM_Avg',
       'Gen_RPM_Std', 'Gen_Bear_Temp_Avg', 'Gen_Phase1_Temp_Avg',
       'Gen_Phase2_Temp_Avg', 'Gen_Phase3_Temp_Avg',
       'Prod_LatestAvg_ActPwrGen0', 'Prod_LatestAvg_ActPwrGen1',
       'Prod_LatestAvg_ReactPwrGen0', 'Prod_LatestAvg_ReactPwrGen1',
       'Gen_SlipRing_Temp_Avg', 'Gen_Bear2_Temp_Avg', 'Rtr_RPM_Max',
       'Rtr_RPM_Min', 'Rtr_RPM_Avg', 'Grd_RtrInvPhase1_Temp_Avg',
       'Grd_RtrInvPhase2_Temp_Avg', 'Grd_RtrInvPhase3_Temp_Avg', 'Rtr_RPM_Std',
       'Amb_WindSpeed_Max', 'Amb_WindSpeed_Min', 'Amb_WindSpeed_Avg',
       'Amb_WindSpeed_Std', 'Amb_WindDir_Relative_Avg', 'Amb_WindDir_Abs_Avg',
       'Amb_Temp_Avg', 'Amb_WindSpeed_Est_Avg', 'Blds_PitchAngle_Min',
       'Blds_PitchAngle_Max', 'Blds_PitchAngle_Avg', 'Blds_PitchAngle_Std',
       'Cont_Top_Temp_Avg', 'Cont_Hub_Temp_Avg', 'Cont_VCP_Temp_Avg',
       'Cont_VCP_ChokcoilTemp_Avg', 'Cont_VCP_WtrTemp_Avg', 'Nac_Temp_Avg',
       'Nac

In [18]:
gen_features_drop = [
    'Gen_RPM_Max', 'Gen_RPM_Min', 'Gen_Phase1_Temp_Avg', 'Gen_Phase3_Temp_Avg',
    'Amb_WindSpeed_Est_Avg', 'Grd_RtrInvPhase1_Temp_Avg',
    'Grd_RtrInvPhase3_Temp_Avg', 'Rtr_RPM_Max', 'Rtr_RPM_Min',
    'Blds_PitchAngle_Max', 'Blds_PitchAngle_Min',
    'Prod_LatestAvg_ReactPwrGen1', 'Cont_Hub_Temp_Avg', 'Spin_Temp_Avg',
    'Rtr_RPM_Std', 'Rtr_RPM_Avg', 'Cont_VCP_Temp_Avg'
]

In [19]:
gen_bear_features_drop = [
    'Gen_RPM_Max', 'Gen_RPM_Min', 'Gen_Phase1_Temp_Avg', 'Gen_Phase3_Temp_Avg',
    'Amb_WindSpeed_Est_Avg', 'Grd_RtrInvPhase1_Temp_Avg',
    'Grd_RtrInvPhase3_Temp_Avg', 'Rtr_RPM_Max', 'Rtr_RPM_Min',
    'Blds_PitchAngle_Max', 'Blds_PitchAngle_Min',
    'Prod_LatestAvg_ReactPwrGen1', 'Cont_Hub_Temp_Avg', 'Spin_Temp_Avg',
    'Rtr_RPM_Std', 'Rtr_RPM_Avg', 'Cont_VCP_Temp_Avg'
]

In [20]:
hyd_features_drop = [
    'Rtr_RPM_Max', 'Rtr_RPM_Min', 'Blds_PitchAngle_Max', 'Blds_PitchAngle_Min',
    'Blds_PitchAngle_Max', 'Grd_RtrInvPhase3_Temp_Avg', 'Grd_Busbar_Temp_Avg',
    'Amb_WindSpeed_Est_Avg', 'Spin_Temp_Avg', 'Cont_Hub_Temp_Avg',
    'Grd_RtrInvPhase1_Temp_Avg', 'Cont_VCP_Temp_Avg'
]

In [21]:
gearbox_features_drop = [
    'Rtr_RPM_Max', 'Rtr_RPM_Min', 'Grd_RtrInvPhase1_Temp_Avg',
    'Grd_RtrInvPhase3_Temp_Avg', 'Blds_PitchAngle_Min', 'Blds_PitchAngle_Max',
    'Cont_VCP_Temp_Avg', 'Grd_Busbar_Temp_Avg', 'Amb_WindSpeed_Est_Avg',
    'Spin_Temp_Avg'
]

In [22]:
transf_features_drop = [
    'HVTrafo_Phase1_Temp_Avg', 'HVTrafo_Phase3_Temp_Avg', 'Rtr_RPM_Max',
    'Rtr_RPM_Min', 'Grd_RtrInvPhase1_Temp_Avg', 'Grd_RtrInvPhase3_Temp_Avg',
    'Blds_PitchAngle_Min', 'Blds_PitchAngle_Max', 'Amb_WindSpeed_Est_Avg',
    'Spin_Temp_Avg', 'Cont_VCP_Temp_Avg'
]

In [23]:
feat_drop_list = [
    gen_features_drop, hyd_features_drop, gen_bear_features_drop,
    transf_features_drop, gearbox_features_drop
]

In [24]:
for df in range(len(comp_df_list)):
    print(f'A actualizar dataframe {df+1} de {len(comp_df_list)}')
#     for col in feat_drop_list:
#         print(f'A apagar a coluna {col}')
    comp_df_list[df] = comp_df_list[df].drop(columns=feat_drop_list[df])

A actualizar dataframe 1 de 5
A actualizar dataframe 2 de 5
A actualizar dataframe 3 de 5
A actualizar dataframe 4 de 5
A actualizar dataframe 5 de 5


In [25]:
for i in comp_df_list:
    print(i.shape)

(434147, 36)
(434145, 28)
(434145, 36)
(434145, 33)
(434145, 31)


In [26]:
for i in comp_df_list:
    print(i.keys())

Index(['Timestamp', 'Turbine_ID', 'Gen_RPM_Avg', 'Gen_RPM_Std',
       'Gen_Bear_Temp_Avg', 'Gen_Phase2_Temp_Avg', 'Prod_LatestAvg_ActPwrGen0',
       'Prod_LatestAvg_ActPwrGen1', 'Prod_LatestAvg_ReactPwrGen0',
       'Gen_SlipRing_Temp_Avg', 'Gen_Bear2_Temp_Avg',
       'Grd_RtrInvPhase2_Temp_Avg', 'Amb_WindSpeed_Max', 'Amb_WindSpeed_Min',
       'Amb_WindSpeed_Avg', 'Amb_WindSpeed_Std', 'Amb_WindDir_Relative_Avg',
       'Amb_WindDir_Abs_Avg', 'Amb_Temp_Avg', 'Blds_PitchAngle_Avg',
       'Blds_PitchAngle_Std', 'Cont_Top_Temp_Avg', 'Cont_VCP_ChokcoilTemp_Avg',
       'Cont_VCP_WtrTemp_Avg', 'Nac_Temp_Avg', 'Nac_Direction_Avg',
       'Grd_Busbar_Temp_Avg', 'Hyd_Oil_Temp_Avg', 'Component', 'TTF',
       '60_days', '45_days', '40_days', '30_days', '10_days', '5_days'],
      dtype='object')
Index(['Timestamp', 'Turbine_ID', 'Hyd_Oil_Temp_Avg', 'Rtr_RPM_Avg',
       'Grd_RtrInvPhase2_Temp_Avg', 'Rtr_RPM_Std', 'Amb_WindSpeed_Max',
       'Amb_WindSpeed_Min', 'Amb_WindSpeed_Avg', 'Amb_Win

# Train and test split

In [27]:
# Preparar a lista de X_train
df_train = [0, 0, 0, 0, 0]
df_test = [0, 0, 0, 0, 0]

for i in range(len(comp_df_list)):
    df_train[i] = dprep.prepare_train_df(comp_df_list[i])
    df_test[i] = dprep.prepare_test_df(comp_df_list[i])

In [28]:
for i in range(len(df_train)):
    print(f'x_train {i} com a forma {df_train[i].shape}')
    print(f'x_test {i} com a forma {df_test[i].shape}')

x_train 0 com a forma (368857, 36)
x_test 0 com a forma (65290, 36)
x_train 1 com a forma (368855, 28)
x_test 1 com a forma (65290, 28)
x_train 2 com a forma (368855, 36)
x_test 2 com a forma (65290, 36)
x_train 3 com a forma (368855, 33)
x_test 3 com a forma (65290, 33)
x_train 4 com a forma (368855, 31)
x_test 4 com a forma (65290, 31)


In [30]:
comp_df_list

[                Timestamp Turbine_ID  Gen_RPM_Avg  Gen_RPM_Std  \
 0     2016-01-01 00:00:00        T11       1270.9         18.8   
 1     2016-01-01 00:10:00        T11       1303.6         40.2   
 2     2016-01-01 00:20:00        T11       1344.3         27.2   
 3     2016-01-01 00:30:00        T11       1377.5         48.9   
 4     2016-01-01 00:40:00        T11       1396.2         66.8   
 ...                   ...        ...          ...          ...   
 87202 2017-08-31 23:20:00        T07       1250.0          8.6   
 87203 2017-08-31 23:30:00        T07       1251.8          8.0   
 87204 2017-08-31 23:40:00        T07       1250.7          7.9   
 87205 2017-08-31 23:50:00        T07       1253.0         12.6   
 87206 2017-09-01 00:00:00        T07       1251.9          9.6   
 
        Gen_Bear_Temp_Avg  Gen_Phase2_Temp_Avg  Prod_LatestAvg_ActPwrGen0  \
 0                   42.0                 61.0                        0.0   
 1                   43.0               

In [31]:
df_train_generator   = df_train[0]
df_train_hydraulic   = df_train[1]
df_train_gen_bear    = df_train[2]
df_train_transformer = df_train[3]
df_train_gearbox     = df_train[4]



In [40]:

df_test_generator   = df_test[0]
df_test_hydraulic   = df_test[1]
df_test_gen_bear    = df_test[2]
df_test_transformer = df_test[3]
df_test_gearbox     = df_test[4]

In [38]:
df_train_generator

,Timestamp,Turbine_ID,Gen_RPM_Avg,Gen_RPM_Std,Gen_Bear_Temp_Avg,Gen_Phase2_Temp_Avg,Prod_LatestAvg_ActPwrGen0,Prod_LatestAvg_ActPwrGen1,Prod_LatestAvg_ReactPwrGen0,Gen_SlipRing_Temp_Avg,...,Grd_Busbar_Temp_Avg,Hyd_Oil_Temp_Avg,Component,TTF,60_days,45_days,40_days,30_days,10_days,5_days
0,2016-01-01 00:00:00,T11,1270.9,18.8,42.0,61.0,0.0,41059.0,0.0,27.0,...,39.0,30.0,0.0,63.0,0,0,0,0,0,0
1,2016-01-01 00:10:00,T11,1303.6,40.2,43.0,61.0,0.0,50808.0,0.0,27.0,...,39.0,30.0,0.0,63.0,0,0,0,0,0,0
2,2016-01-01 00:20:00,T11,1344.3,27.2,43.0,61.0,0.0,60667.0,0.0,27.0,...,39.0,30.0,0.0,63.0,0,0,0,0,0,0
3,2016-01-01 00:30:00,T11,1377.5,48.9,43.0,62.0,0.0,77486.0,0.0,27.0,...,39.0,30.0,0.0,63.0,0,0,0,0,0,0
4,2016-01-01 00:40:00,T11,1396.2,66.8,43.0,62.0,0.0,67378.0,0.0,27.0,...,40.0,30.0,0.0,63.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74167,2017-05-31 23:10:00,T07,1349.3,76.8,69.0,103.0,0.0,63761.0,0.0,41.0,...,44.0,43.0,0.0,82.0,0,0,0,0,0,0
74168,2017-05-31 23:20:00,T07,1329.5,71.5,67.0,98.0,0.0,55821.0,0.0,39.0,...,42.0,42.0,0.0,82.0,0,0,0,0,0,0
74169,2017-05-31 23:30:00,T07,1339.7,89.6,65.0,93.0,0.0,52651.0,0.0,38.0,...,40.0,43.0,0.0,82.0,0,0,0,0,0,0
74170,2017-05-31 23:40:00,T07,1479.7,89.2,63.0,90.0,0.0,85663.0,0.0,38.0,...,39.0,44.0,0.0,82.0,0,0,0,0,0,0


In [39]:
def bin_classify(model, clf, features,target, df_train, df_test ,params=None, score=None, ):
    
    """Perfor Grid Search hyper parameter tuning on a classifier.
    
    Args:
        model (str): The model name identifier
        clf (clssifier object): The classifier to be tuned
        features (list): The set of input features names
        params (dict): Grid Search parameters
        score (str): Grid Search score
        
    Returns:
        Tuned Clssifier object
        dataframe of model predictions and scores
    
    """
    
    
    X_train = df_train[features]

    X_test = df_test[features] 
    y_train = df_train[target]
    y_test = df_test[target]

    
    grid_search = model_selection.GridSearchCV(estimator=clf, param_grid=params, cv=5, scoring=score, n_jobs=-1)

    grid_search.fit(X_train, y_train)
    y_pred = grid_search.predict(X_test)
    
    if hasattr(grid_search, 'predict_proba'):   
        y_score = grid_search.predict_proba(X_test)[:,1]
    elif hasattr(grid_search, 'decision_function'):
        y_score = grid_search.decision_function(X_test)
    else:
        y_score = y_pred
        
    predictions = {'y_pred' : y_pred, 'y_score' : y_score}
    df_predictions = pd.DataFrame.from_dict(predictions)
    
    return grid_search.best_estimator_, df_predictions